In [1]:
#using Revise

In [2]:
# Add PATH
push!(LOAD_PATH, joinpath(@__DIR__, "..", "src"))

#using PyPlot
using Plots
using BenchmarkTools
using DifferentialEquations

using LSM.PhysCon
using LSM.WaterMod
#using LSM.LeafPhotosynthesisMod
using LSM.LeafEnergyWaterBalanceMod

output_dir = joinpath(@__DIR__,"..","output")
mkpath(output_dir)

"/Users/gentine/Dropbox/Collaborations_Projects/Optimal_Stomata/LSM-SPAM/test/../output"

In [3]:
using DelimitedFiles
PAR  = readdlm("PAR.dat");
size(PAR)

(172422, 1)

In [ ]:
tmax = length(PAR) # Time in seconds here
t = range(1,tmax,step=1)
l = leaf_params{Float32}()
f = fluxes{Float32}()
met = meteo{Float32}()

mutable struct parameters_ode
    l::leaf_params;
    met::meteo;
    f::fluxes;
    psi_s;
end

function f_ode!(du,u,p,t) # p are parameters
    du .= LeafEnergyWaterBalance(u[1], u[2], u[3], p.met, p.l, p.f, p.psi_s);
    #println("du_inside = $(du), u_inside = $(u)")
end


f.Je   = 100; f.gbc  = 100; f.gbv  = 100; f.ceair= 1500; f.eair = 1500;  f.H=0;f.LE=0; # leaf should not have eair
l.Kn = 2.44; l.α=0.2; l.ε=0.98; l.LMA=100e-3; l.RWC=80/100;l.psi_l=-1e6;l.psi_l50 = -2e6;l.ck=3;met.zscreen = 2.0;
l.height   = 1.0; 
met.stab_type_stable = 2;
psi_s      = -0.3e6 ; # soil water potential (Pa)
U          =  1.0;
RH         =  65/100;
eps_air    =  0.75;

l.dynamic_state = true
l.gstyp = 1 # Ball Berry
l.g1_BB=4
met.Ca = 400
l.Vcmax25 = 90
l.Jmax25 = 90*1.9
uu = zeros(tmax,16)
uu[1,2] = 0.07
uu[1,1] = 0.5
f.APAR = 50
l.Rdleaf = 1.0
l.T = met.T_air = 298;
l.psi_l = psi_s;
l.Cc = 0.6*400;
#apar[1:500]=0
#apar[501:100]=0
tspan      =  (0.0,1.0);



for c = 2:1:tmax
    #println(apar[c])
       met.S_down = PAR[c]/(45/100*physcon.Wtoμmole_s);
       met.T_air  = 298.0;
       met.L_down = eps_air*physcon.σ*(met.T_air)^4;
       met.e_air  = RH*SatVap(met.T_air)[1];
       met.PAR    = PAR[c];
    
       f.APAR = PAR[c]*0.5 
       f.APAR = max(10,f.APAR)
       #l.gs = uu[c-1,2]
       l.Kn = uu[c-1,1]
       uu[c-1,3]=l.ϕs
       uu[c-1,4]=l.Cc
       uu[c-1,5]=f.φ
       uu[c-1,6]=l.Kp
       uu[c-1,7]=f.APAR
       uu[c-1,8]=f.An_biochemistry
       uu[c-1,9]=l.Fm′
       uu[c-1,10]=l.T
       uu[c-1,11]=f.Rn
       uu[c-1,12]=f.H
       uu[c-1,13]=f.LE
       uu[c-1,14]=l.Cc
       uu[c-1,15]=f.An_diffusion
       uu[c-1,16]=l.gs
#        println("gs out=",l.gs)
       #l.Kp = 4
       u    = [l.T;l.psi_l;l.Cc];
       p    = parameters_ode(l,met,f,psi_s);
        #(p.met, p.l,  p.psi_s, p.U)    = [l;met;psi_s;U];
        #prob = ODEProblem(f_ode!,u0,tspan,p);
        #du   = zeros(size(u));
        #f_ode!(du,u,p,t,dummy);
        #println("du_outside = $(du), u_outside = $(u)")
        # Rn_t[i] = dummy[3]; H_t[i] = dummy[4]; LE_t[i] = dummy[5];


#             du   = zeros(size(u));
#             f_ode!(du,u,p,t);
#             (l.T,l.psi_l,l.Cc) = du*dt+u;

       u0   = [l.T;l.psi_l;l.Cc];
       prob = ODEProblem(f_ode!,u0,tspan,p);
       sol  = solve(prob);            
       # save values
       met = p.met;
       f   = p.f;
#        println("gs out 1=",l.gs)
       l   = p.l;
#        println("gs out 2=",l.gs)
       (l.T,l.psi_l,l.Cc) = sol[1:3,end];
            
    
       #LeafPhotosynthesis!(f,l,met)
    if (l.Kn_ss-l.Kn) > 0
        tau_k = 1
    else
        tau_k = 1
    end
       uu[c,1] = uu[c-1,1]+(l.Kn_ss-l.Kn)/tau_k*1/60
       uu[c,2] = uu[c-1,2]+(l.gs_ss-l.gs)/20*1/60
    #println(l.Kn_ss, " ",  l.Ci, " ", f.φ, " ", l.gs_ss, " ", f.APAR)
end

Cs=397.5601, ra=10.0, ra/rleaf=0.015249534, Cc=240.0, Ca=400.0 L=-4.214819e-12 u*=5.3969998e-6 H=0.0
gs=0.122283936, Cs=397.5601 An_biochemistry=3.4073856 RH=0.65
397.5601 ppm, 1.09934619140625 (kPA), 3.4073856 micromol/s/m2   An_diffusion=9.847506Cs=397.10373, ra=10.0, ra/rleaf=0.018584574, Cc=244.15656, Ca=400.0 L=2.717856e-12 u*=4.8026945e-6 H=-8.104434
gs=0.122680575, Cs=397.10373 An_biochemistry=3.4498205 RH=0.6526818
397.10373 ppm, 1.08643994140625 (kPA), 3.4498205 micromol/s/m2   An_diffusion=11.6893635Cs=397.017, ra=10.0, ra/rleaf=0.018643731, Cc=240.0, Ca=400.0 L=-3.6468753e-12 u*=5.4954085e-6 H=0.0
gs=0.122314416, Cs=397.017 An_biochemistry=3.4073856 RH=0.65
397.017 ppm, 1.09934619140625 (kPA), 3.4073856 micromol/s/m2   An_diffusion=12.039335Cs=397.0282, ra=10.0, ra/rleaf=0.01858912, Cc=240.13213, Ca=400.0 L=-3.7849533e-12 u*=5.469984e-6 H=-0.20081785
gs=0.12232461, Cs=397.0282 An_biochemistry=3.40869 RH=0.6500663
397.0282 ppm, 1.099025634765625 (kPA), 3.40869 micromol/s/m2  

gs=0.13687015, Cs=243.79906 An_biochemistry=3.4454882 RH=0.6522229
243.79906 ppm, 1.08864111328125 (kPA), 3.4454882 micromol/s/m2   An_diffusion=0.00015033643Cs=396.75677, ra=10.0, ra/rleaf=0.020755358, Cc=243.74069, Ca=400.0 L=4.7243177e-12 u*=4.808707e-6 H=-6.9712477
gs=0.12265752, Cs=396.75677 An_biochemistry=3.4452868 RH=0.652306
396.75677 ppm, 1.088241943359375 (kPA), 3.4452868 micromol/s/m2   An_diffusion=13.089583Cs=243.7669, ra=4.193427e7, ra/rleaf=0.9999874, Cc=243.76494, Ca=400.0 L=-1.0585959e-11 u*=1.5442423e-7 H=-1.5623694e-6
gs=0.13686723, Cs=243.7669 An_biochemistry=3.4450572 RH=0.652167
243.7669 ppm, 1.0889091796875 (kPA), 3.4450572 micromol/s/m2   An_diffusion=0.00015036737Cs=396.74594, ra=10.0, ra/rleaf=0.020754924, Cc=243.21513, Ca=400.0 L=1.9516437e-11 u*=4.8174984e-6 H=-5.296571
gs=0.12259952, Cs=396.74594 An_biochemistry=3.4392984 RH=0.6517512
396.74594 ppm, 1.09090625 (kPA), 3.4392984 micromol/s/m2   An_diffusion=13.133334Cs=243.93706, ra=4.193427e7, ra/rleaf=0.99

gs=0.13766186, Cs=231.2138 An_biochemistry=3.338712 RH=0.65204346
231.2138 ppm, 1.08950244140625 (kPA), 3.338712 micromol/s/m2   An_diffusion=0.00016244917Cs=397.0584, ra=10.0, ra/rleaf=0.020872911, Cc=259.07172, Ca=400.0 L=1.1199529e-12 u*=4.7710814e-6 H=-13.90305
gs=0.123550214, Cs=397.0584 An_biochemistry=3.5711477 RH=0.65460825
397.0584 ppm, 1.077234375 (kPA), 3.5711477 micromol/s/m2   An_diffusion=11.87219Cs=396.50677, ra=10.0, ra/rleaf=0.018773409, Cc=213.92686, Ca=400.0 L=-5.4506395e-12 u*=5.22651e-6 H=-1.8468069
gs=0.12082979, Cs=396.50677 An_biochemistry=3.1736193 RH=0.65060997
396.50677 ppm, 1.096401123046875 (kPA), 3.1736193 micromol/s/m2   An_diffusion=14.098617Cs=398.0107, ra=10.0, ra/rleaf=0.018367633, Cc=291.69647, Ca=400.0 L=5.2734434e-13 u*=4.7185395e-6 H=-22.96854
gs=0.12511702, Cs=398.0107 An_biochemistry=3.8003135 RH=0.6576329
398.0107 ppm, 1.0628896484375 (kPA), 3.8003135 micromol/s/m2   An_diffusion=8.028705Cs=244.03114, ra=4.193427e7, ra/rleaf=0.9999876, Cc=244.0

396.7761 ppm, 1.08228125 (kPA), 3.4571264 micromol/s/m2   An_diffusion=13.011641Cs=244.74525, ra=4.193427e7, ra/rleaf=0.99998754, Cc=244.74332, Ca=400.0 L=-1.0964872e-11 u*=1.5442423e-7 H=-2.5021855e-6
gs=0.1369211, Cs=244.74525 An_biochemistry=3.457006 RH=0.6534745
244.74525 ppm, 1.0826455078125 (kPA), 3.457006 micromol/s/m2   An_diffusion=0.00014942576Cs=396.77014, ra=10.0, ra/rleaf=0.020762924, Cc=244.44148, Ca=400.0 L=2.0503217e-12 u*=4.793538e-6 H=-9.811386
gs=0.122746035, Cs=396.77014 An_biochemistry=3.453874 RH=0.6532482
396.77014 ppm, 1.083727783203125 (kPA), 3.453874 micromol/s/m2   An_diffusion=13.035629Cs=244.8376, ra=4.193427e7, ra/rleaf=0.9999874, Cc=244.83565, Ca=400.0 L=-1.0933895e-11 u*=1.5442423e-7 H=-2.427787e-6
gs=0.13690583, Cs=244.8376 An_biochemistry=3.4574294 RH=0.65337086
244.8376 ppm, 1.08314111328125 (kPA), 3.4574294 micromol/s/m2   An_diffusion=0.00014933688Cs=396.76254, ra=10.0, ra/rleaf=0.02076066, Cc=244.05757, Ca=400.0 L=1.7785056e-12 u*=4.78944e-6 H=-10.

396.77243 ppm, 1.079896240234375 (kPA), 3.4718194 micromol/s/m2   An_diffusion=13.026382Cs=267.4573, ra=4.193427e7, ra/rleaf=0.99998754, Cc=267.45566, Ca=400.0 L=-1.19050785e-11 u*=1.5442423e-7 H=-4.576793e-6
gs=0.13583677, Cs=267.4573 An_biochemistry=3.6506803 RH=0.6563711
267.4573 ppm, 1.068857666015625 (kPA), 3.6506803 micromol/s/m2   An_diffusion=0.00012756641Cs=396.81302, ra=10.0, ra/rleaf=0.020601884, Cc=245.30652, Ca=400.0 L=1.2993204e-12 u*=4.778207e-6 H=-12.619249
gs=0.12292798, Cs=396.81302 An_biochemistry=3.4769113 RH=0.6541812
396.81302 ppm, 1.0792705078125 (kPA), 3.4769113 micromol/s/m2   An_diffusion=12.862596Cs=245.51466, ra=4.193427e7, ra/rleaf=0.99998754, Cc=245.51274, Ca=400.0 L=-1.1283405e-11 u*=1.5442423e-7 H=-3.2436055e-6
gs=0.13710432, Cs=245.51466 An_biochemistry=3.4795809 RH=0.65450805
245.51466 ppm, 1.0777119140625 (kPA), 3.4795809 micromol/s/m2   An_diffusion=0.00014868523Cs=396.78687, ra=10.0, ra/rleaf=0.02079013, Cc=245.44962, Ca=400.0 L=1.1204247e-12 u*=4.7

...

397.28952 ppm, 0.977830810546875 (kPA), 3.7008762 micromol/s/m2   An_diffusion=10.939468Cs=258.04782, ra=4.193427e7, ra/rleaf=0.9999878, Cc=258.04608, Ca=400.0 L=-2.8879243e-11 u*=1.5442423e-7 H=-1.8834804e-5
gs=0.13785137, Cs=258.04782 An_biochemistry=3.6086254 RH=0.67667484
258.04782 ppm, 0.97552685546875 (kPA), 3.6086254 micromol/s/m2   An_diffusion=0.00013662264Cs=396.95758, ra=10.0, ra/rleaf=0.020901047, Cc=254.43759, Ca=400.0 L=7.8902156e-14 u*=4.118162e-6 H=-88.03352
gs=0.12459974, Cs=396.95758 An_biochemistry=3.591098 RH=0.67980975
396.95758 ppm, 0.96161328125 (kPA), 3.591098 micromol/s/m2   An_diffusion=12.279112Cs=258.04785, ra=4.193427e7, ra/rleaf=0.9999876, Cc=258.04608, Ca=400.0 L=-2.8879246e-11 u*=1.5442423e-7 H=-1.8834804e-5
gs=0.13785137, Cs=258.04785 An_biochemistry=3.6086254 RH=0.67667484
258.04785 ppm, 0.97552685546875 (kPA), 3.6086254 micromol/s/m2   An_diffusion=0.00013662262Cs=397.03192, ra=10.0, ra/rleaf=0.020901047, Cc=257.99426, Ca=400.0 L=9.665473e-14 u*=4.242

258.53827 ppm, 0.97368798828125 (kPA), 3.6133966 micromol/s/m2   An_diffusion=0.00013615061Cs=397.04187, ra=10.0, ra/rleaf=0.020901227, Cc=258.4708, Ca=400.0 L=9.3860703e-14 u*=4.2262172e-6 H=-80.42037
gs=0.12464962, Cs=397.04187 An_biochemistry=3.6131635 RH=0.6771718
397.04187 ppm, 0.9733125 (kPA), 3.6131635 micromol/s/m2   An_diffusion=11.938989Cs=258.51642, ra=4.193427e7, ra/rleaf=0.9999876, Cc=258.51468, Ca=400.0 L=-2.962486e-11 u*=1.5442423e-7 H=-1.9087074e-5
gs=0.13785008, Cs=258.51642 An_biochemistry=3.613103 RH=0.6770404
258.51642 ppm, 0.973897705078125 (kPA), 3.613103 micromol/s/m2   An_diffusion=0.00013617161Cs=397.03284, ra=10.0, ra/rleaf=0.020900853, Cc=258.03665, Ca=400.0 L=9.659263e-14 u*=4.24262e-6 H=-79.14016
gs=0.12460375, Cs=397.03284 An_biochemistry=3.608715 RH=0.6767294
397.03284 ppm, 0.97528369140625 (kPA), 3.608715 micromol/s/m2   An_diffusion=11.975397Cs=258.71283, ra=4.193427e7, ra/rleaf=0.9999876, Cc=258.71106, Ca=400.0 L=-2.9225844e-11 u*=1.5442423e-7 H=-1.895

258.5822 ppm, 0.97111083984375 (kPA), 3.615388 micromol/s/m2   An_diffusion=0.00013610831Cs=397.04285, ra=10.0, ra/rleaf=0.020908175, Cc=258.5653, Ca=400.0 L=9.081027e-14 u*=4.2068127e-6 H=-81.89065
gs=0.1246828, Cs=397.04285 An_biochemistry=3.615321 RH=0.6776804
397.04285 ppm, 0.971050048828125 (kPA), 3.615321 micromol/s/m2   An_diffusion=11.934982Cs=258.73627, ra=4.193427e7, ra/rleaf=0.9999876, Cc=258.7345, Ca=400.0 L=-3.1057146e-11 u*=1.5442423e-7 H=-1.9537742e-5
gs=0.13789095, Cs=258.73627 An_biochemistry=3.6165888 RH=0.677694
258.73627 ppm, 0.9709892578125 (kPA), 3.6165888 micromol/s/m2   An_diffusion=0.00013596006Cs=397.04202, ra=10.0, ra/rleaf=0.02090692, Cc=258.51675, Ca=400.0 L=9.056808e-14 u*=4.205224e-6 H=-82.00899
gs=0.124682724, Cs=397.04202 An_biochemistry=3.6150842 RH=0.6777213
397.04202 ppm, 0.970867919921875 (kPA), 3.6150842 micromol/s/m2   An_diffusion=11.938363Cs=267.26285, ra=4.193427e7, ra/rleaf=0.9999876, Cc=267.2612, Ca=400.0 L=-3.05306e-11 u*=1.5442423e-7 H=-1.9

Excessive output truncated after 524296 bytes.

, ra=10.0, ra/rleaf=0.020909477, Cc=258.71246, Ca=400.0 L=8.855236e-14 u*=4.1916655e-6 H=-83.005905
gs=0.12471157, Cs=397.04575 An_biochemistry=3.6175015 RH=0.6780664
397.04575 ppm, 0.9693349609375 (kPA), 3.6175015 micromol/s/m2   An_diffusion=11.923306Cs=266.78247, ra=4.193427e7, ra/rleaf=0.9999876, Cc=266.78082, Ca=400.0 L=-3.1321484e-11 u*=1.5442423e-7 H=-1.9616416e-5
gs=0.13733834, Cs=266.78247 An_biochemistry=3.6740537 RH=0.67780817
266.78247 ppm, 0.97048193359375 (kPA), 3.6740537 micromol/s/m2   An_diffusion=0.00012821592Cs=397.06192, ra=10.0, ra/rleaf=0.02082488, Cc=258.91495, Ca=400.0 L=8.843273e-14 u*=4.190975e-6 H=-83.05611
gs=0.12472159, Cs=397.06192 An_biochemistry=3.619022 RH=0.6780838
397.06192 ppm, 0.969257568359375 (kPA), 3.619022 micromol/s/m2   An_diffusion=11.8580475Cs=258.9817, ra=4.193427e7, ra/rleaf=0.9999876, Cc=258.97995, Ca=400.0 L=-3.2224747e-11 u*=1.5442423e-7 H=-1.9875528e-5
gs=0.13791591, Cs=258.9817 An_biochemistry=3.619785 RH=0.6781844
258.9817 ppm, 0.968

In [ ]:
plot(t/3600, uu[:,8],label="GPP")
plot!(t/3600, PAR/10,label="PAR/10")

In [ ]:
plot(t/3600, uu[:,16],label="g_leaf")

In [ ]:
plot(t/3600, uu[:,8],label="GPP biochemistry")
plot!(t/3600, uu[:,15],label="GPP diffusion")

In [ ]:
plot(t/3600, uu[:,11],label="Rn",ylim=-50:500)
plot!(t/3600, uu[:,12],label="H",ylim=-50:500)
plot!(t/3600, uu[:,13],label="LE",ylim=-50:500)

In [ ]:
plot(t/3600, uu[:,14]/met.Ca,label="Cc/Ca")